## 前置知识
在运行此脚本之前， 请先查阅我们编写的文档: `第0章. 概述` / `Gemini2连接与USB规则配置`。


```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```


## 导入依赖

In [2]:
import time
import sys
import os

# 导入阿凯写的Orbbec工具库
# 确保orbbec_utils.py跟你目前所执行的脚本在同一级目录下
from orbbecsdk_utils import *
# 添加Python Path
add_path_pyorbbecsdk()

# 导入pyorbbecsdk
from pyorbbecsdk import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 配置日志等级

In [3]:
# 设置日志等级为ERROR 
# 这样不会频繁的打印日志信息
ctx = Context()
ctx.set_logger_level(OBLogLevel.ERROR)

[06/24 13:36:20.063292][info][16495][Context.cpp:68] Context created with config: default config!
[06/24 13:36:20.063327][info][16495][Context.cpp:73] Work directory=/home/kyle/Project/奥比中光3D相机/Gemini335-软件资料-阿凯爱玩机器人/02.奥比中光-pyobbecsdk示例代码(Gemini335), SDK version=v1.10.5-20240511-7dc8be9
[06/24 13:36:20.063343][info][16495][LinuxPal.cpp:31] createObPal: create LinuxPal!
[06/24 13:36:20.137100][info][16495][LinuxPal.cpp:142] Create PollingDeviceWatcher!
[06/24 13:36:20.137118][info][16495][DeviceManager.cpp:15] Current found device(s): (1)
[06/24 13:36:20.137122][info][16495][DeviceManager.cpp:24] 	- Name: Orbbec Gemini 335, PID: 0x0800, SN/ID: CP1L44P00049, Connection: USB3.2


通过日志信息, 我们可以看到使用的是USB3.0的端口(` Connection: USB3.0`)。

## 查看相机列表

In [4]:
# 查询设备列表 
device_list = ctx.query_devices()
# 获取设备个数
device_num = device_list.get_count()

if device_num == 0:
    print("[ERROR]没有设备连接")
else:
    print(f"检测到{device_num}个设备")

检测到1个设备


**注意事项**
1. 如果检测到当前没有设备， 就不要去执行后面的`连接设备`的操作了。会导致程序异常退出。
2. 如果你明明将相机插入到电脑上，但是这里并没有设备连接的显示， 你需要参考我们编写的文档: `第0章. 概述` / `Gemini335连接与USB规则配置`



## 连接设备

获取第一个设备的序列号

In [5]:
# 获取特定索引下的设备序列号
serial_num = device_list.get_device_serial_number_by_index(0)
print(f"设备序列号为: {serial_num}")

设备序列号为: CP1L44P00049


需要注意的是相机属于USB设备，USB设备有独占属性， 不能够同时被多个进程/脚本访问。
在连接设备之前，需要关闭其他占用设备的脚本/上位机软件。

而且设备在连接之后，不能重复连接。重新连接之前需要先释放当前的连接。 

可以通过索引号连接设备。

In [5]:
# 获取第一个设备, 并打开设备
# device = device_list.get_device_by_index(0)

也可以根据序列号连接设备

In [6]:
# 根据设备序列号创建设备
# device = device_list.get_device_by_serial_number(serial_num)

阿凯推荐用阿凯写的orbbec工具箱里面的设备连接方法, 具体实现比较复杂， 细节见`orbbecsdk_utils.py`

In [6]:
# 连接设备
ret, device = connect_device()
print(f"[INFO] 设备是否连接成功: {ret}")

[INFO] 设备是否连接成功: True


In [8]:
# 下面是指定序列号的写法
# ret, device = connect_device(serial_num=serial_num)

**注意事项**
* 如果设备创建失败，就不要继续后续的操作了。

## 获取相机信息

需要注意的是，在获取设备的时候，同时也就打开了。 
USB设备是独占性的，只能被一个脚本/软件使用。因此在执行下面的代码的时候，注意需要把其他占用相机的上位机/Jupyter Notebook脚本/Python脚本都关掉。 
而且如果打开了一次， 就不能重复打开。 需要先释放对应的资源。

In [7]:
# 获取设备信息
device_info = device.get_device_info()
# 可以直接将设备信息打印出来
print(device_info)

DeviceInfo(name=Orbbec Gemini 335
, pid=2048
, vid=11205, uid=2-5-7
, serial_number=CP1L44P00049
, firmware_version=1.2.20
, connection_type=USB3.2
, hardware_version=0.1
, supported_min_sdk_version=1.0.1
, device_type=1)


In [8]:
device_name = device_info.get_name()
print(f"设备名称: {device_name}")

设备名称: Orbbec Gemini 335


In [9]:
connec_type = device_info.get_connection_type()
print(f"设备连接方式: {connec_type}")

if connec_type not in  ["USB3.0", "USB3.1", "USB3.2"]:
    print("请使用USB3.0端口以及USB3.0的数据线")

设备连接方式: USB3.2


In [12]:
# 获取设备类型
device_info.get_device_type()

<OBDeviceType.LIGHT_BINOCULAR: 1>

奥比中光的3D相机的USB设备公司ID(`VID`)均为`0x2bc5`

In [10]:
device_vid = device_info.get_vid()
print(f"USB设备公司ID: {device_vid}  十六进制: {hex(device_vid)}")

USB设备公司ID: 11205  十六进制: 0x2bc5


相机的不同型号有各自对应的`PID`, 例如Gemini355对应的`PID`为`0x0670`

In [12]:
device_pid = device_info.get_pid()
print(f"产品型号ID: {device_pid}  十六进制: {hex(device_pid)}")

产品型号ID: 2048  十六进制: 0x800


比如你手里有两个Gemini355, 他们的`VID`跟`PID`都是一样的。 那么如何去获取某一个特定的设备呢？
这个时候就需要通过设备的序列号来辨识， 每一个设备都有一个独一无二的序列号。

In [15]:
# 获取设备的序列号
device_info.get_serial_number()

'CP1L44P00049'

获取相机的`UID`， `UID`记录了USB设备在USB总线挂载的物理路径。简单来说就是插到设备的哪个USB口上了。
实际上物理挂载链路也可以用于辨识唯一的设备， 前提是相机固定在一个特定的USB口上。

In [13]:
device_info.get_uid()

'2-5-7'

获取相机硬件的版本号

In [14]:
print("相机硬件版本号: ")
print(device_info.get_hardware_version())

相机硬件版本号: 
0.1


获取相机固件的版本号

In [15]:
print("相机固件版本号: ")
print(device_info.get_firmware_version())

相机固件版本号: 
1.2.20


获取支持的Orbbec SDK最低版本

In [16]:
print("SDK最低支持版本")
print(device_info.get_supported_min_sdk_version())

SDK最低支持版本
1.0.1


## 获取基线信息

In [17]:
# 获取设备的基线
baseline = device.get_baseline()

In [18]:
# 基线数据 (两个红外相机等效光心的直线距离)
print(f"基线为: {baseline.baseline} mm")

基线为: 50.50199890136719 mm


In [19]:
# TODO 两个红外相机等效光心在Z轴上的偏移量 ？
baseline.zpd

0.0

## 相机设备状态

In [23]:
# # 这个API调不通
# # TODO [ERROR] 报错 RuntimeError: Caught an unknown exception!
# device_state = device.get_device_state()

## 与设备重新创建连接
将设备断开连接并重新创建连接。在设备断开连接之后，之前的设备`device`对象的访问可能会异常。
所以在设备reboot之后，需要删除这个`device`对象。 然后在重新创建连接。

In [20]:
# 断开设备, 重新创建连接
device.reboot()
# 删除设备
del device

In [21]:
# 重新查询设备列表 
device_list = ctx.query_devices()
if device_list.get_count() != 0:
    # 根据设备序列号创建设备
    device = device_list.get_device_by_serial_number(serial_num)

In [22]:
print(device)